In [16]:
%pip install -U langchain-tavily

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.environ["TAVILY_API_KEY"] = "YOUR_TAVILY_API_KEY"

In [18]:
from typing import Annotated, List, Dict, Any
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from typing_extensions import TypedDict
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model = "gpt-4o",
    temperature = 0.7
)

In [20]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [21]:
graph = StateGraph(State)

In [22]:
tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

In [23]:
def chatbot(state: State) -> Dict[str, Any]:
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
graph.add_node("chatbot", chatbot)

In [24]:
tool_node = ToolNode(tools=[tool])
graph.add_node("tools", tool_node)

In [25]:
graph.add_conditional_edges(
    "chatbot",
    tools_condition,
)

In [26]:
graph.add_edge("tools", "chatbot")
graph.add_edge(START, "chatbot")
app = graph.compile()

In [27]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e)

KeyboardInterrupt: 

In [28]:
def test(user_input):
    state: State = {
        "messages": [HumanMessage(content=user_input)],
    }
    state = app.invoke(state)
    print("\n--- Response 1 ---")
    for m in state["messages"]:
        print(m.__class__.__name__, ":", m.content)

In [29]:
test("What is the cricket score?")


--- Response 1 ---
HumanMessage : What is the cricket score?
AIMessage : 
ToolMessage : {"query": "cricket score", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.theguardian.com/sport/2025/sep/03/theyre-like-time-machines-the-endless-magic-of-cricket-scorecards", "title": "‘They’re like time machines’: the endless magic of cricket scorecards - The Guardian", "score": 0.47256237, "published_date": "Wed, 03 Sep 2025 23:08:00 GMT", "content": "# ‘They’re like time machines’: the endless magic of cricket scorecards Bill Frindall, the BBC’s encyclopaedic scorer nicknamed “Bearders”, went further: “The scorecard is cricket’s autobiography, written one ball at a time.” For Frindall, who recorded every Test between 1966 and 2008 in immaculate longhand, the card wasn’t a dry ledger but a diary, each entry carrying the cadence of a ball bowled and played. And just like the Matrix code, cricket’s scorecard language has been painstakingly invented. The

In [30]:
test("What is the current news in the news channels? Give me top 3 Headlines")


--- Response 1 ---
HumanMessage : What is the current news in the news channels? Give me top 3 Headlines
AIMessage : 
ToolMessage : {"query": "current news headlines", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.skynews.com.au/australia-news/todays-news-headlines-sebastien-lecornu-appointed-france-pm/video/4638ac6b0ef34241407fe7fd9005c8f8", "title": "Today’s News Headlines: Sebastien Lecornu appointed France PM - Sky News Australia", "score": 0.5717911, "published_date": "Tue, 09 Sep 2025 23:02:04 GMT", "content": "Today’s News Headlines: Sebastien Lecornu appointed France PM | Sky News Australia SKY NEWS LIVE Image 1: Sky News Australia MOST POPULARRussia-China energy deal could have ‘long-term implications’ for Australia’s critical liquified natural gas export industry Sky News Australia Discover Effortless Glucose Monitoring: Request a Free Trial Dexcom | [SponsoredSponsored Click Here](https://www.skynews.com.au/australia-news/todays

In [31]:
test("Hi how are you?")


--- Response 1 ---
HumanMessage : Hi how are you?
AIMessage : Hello! I'm here and ready to assist you. How can I help you today?
